# Keener's Method

Keener's Method states that the `strength` $s$ of a team should 

- consider its interaction with opponents and these opponents' strengths, as well as,
- be proportionally constant to its `rating` $r$
  - $s = \lambda r$.

The strength statistics $a_{ij}$ created by the i-th team when playing the j-th team can be derived from something meaning such as wins, yards, passes, etc... For example, $a_{ij}$ can be the number of wins by the i-th team over the j-th team.

$a_{ij} = w_{ij}$

Or it can consider ties as well.

$a_{ij} = w_{ij} - \dfrac{t_{ij}}{2}$

Whatever strength statistics you use, we can denoted as $S_{ij}$.

$a_{ij} = S_{ij}$

Typically, we want want to normalize $a_{ij}$ as follows to avoid disproportionate effects on the ratings.

$a_{ij} = \dfrac{S_{ij}}{S_{ij} + S_{ji}}$

But, since there are some pairs of teams that might have not played, we can use [Laplace's rule of succession](https://en.wikipedia.org/wiki/Rule_of_succession) to normalize as follows.

$a_{ij} = \dfrac{S_{ij} + 1}{S_{ij} + S_{ji} + 2}$

Keener also suggests that we apply a `skewing function` to combat skew in the data. Skew could come from teams inflating their strength statistics to increase their ratings and thereby ranking. A particular skew function recommended by Keener is as follows.

$h(x) = \dfrac{1}{2} + \dfrac{\mathrm{sgn} \{x - \frac{1}{2} \} \sqrt{|2x - 1|}}{2}$

The strength $s$ of each team is given as

$s = Ar$,

where

- $A$ is a $n$ x $n$ matrix with $a_{ij}$ element representing the strength statistic of the i-th team when it plays the j-th team, and
- $r$ is the rating.

Remember that we stated two things:

- $s = \lambda r$, and
- $s = Ar$.

Thus,

$Ar = \lambda r$,

and we can interpret

- $\lambda$ as an eigenvalue, and
- $r$ as the corresponding eigenvector to the eigenvalue.

Using any numerical method we want, when can compute the eigenvectors and eigenvalues of $A$. The eigenvector associated with the largest eigenvalue will have its elements correspond to the teams and be the ratings. When the ratings are sorted (the values of the eigenvector), then the ranking is produced.

## NFL 2009

Let's look at the whole NFL 2009 season. This data has scores, yards and turnovers. We will use scores to create the strength statistics.

In [1]:
import pandas as pd

def get_nfl_2009():
    return pd.read_csv('./nfl/2009.csv')

df = get_nfl_2009()
teams = sorted(list(set(df.t1) | set(df.t2)))

df

,t1,t2,s1,s2,yd1,yd2,to1,to2,home_team,week
0,Steelers,Titans,13,10,357,320,3,2.0,Steelers,1
1,Broncos,Bengals,12,7,302,307,0,2.0,Bengals,1
2,Ravens,Chiefs,38,24,501,188,1,0.0,Ravens,1
3,Colts,Jaguars,14,12,365,228,2,0.0,Colts,1
4,Cowboys,Buccaneers,34,21,462,450,0,0.0,Buccaneers,1
...,...,...,...,...,...,...,...,...,...,...
251,Cowboys,Eagles,24,0,474,228,1,1.0,Cowboys,17
252,Chiefs,Broncos,44,24,524,512,2,3.0,Broncos,17
253,Ravens,Raiders,21,13,330,325,0,2.0,Raiders,17
254,Titans,Seahawks,17,13,304,309,2,1.0,Seahawks,17


$a_{ij}$ will actually be the total number of points scored by the i-th team on the j-th team during the whole season. These are the cummulative points.

In [2]:
def get_cummulative_score(df, t1, t2, f1='s1', f2='s2'):
    s1 = df[(df.t1 == t1) & (df.t2 == t2)][f1].sum()
    s2 = df[(df.t1 == t2) & (df.t2 == t1)][f2].sum()
    s = s1 + s2
    return s

rscore_df = pd.DataFrame([[get_cummulative_score(df, t1, t2) if t1 != t2 else 0 for t2 in teams] 
              for t1 in teams], index=teams, columns=teams)
rscore_df

,49ers,Bears,Bengals,Bills,Broncos,Browns,Buccaneers,Cardinals,Chargers,Chiefs,...,Raiders,Rams,Ravens,Redskins,Saints,Seahawks,Steelers,Texans,Titans,Vikings
49ers,0,10,0,0,0,0,0,44,0,0,...,0,63,0,0,0,40,0,21,27,24
Bears,6,0,10,0,0,30,0,21,0,0,...,0,17,7,0,0,25,17,0,0,46
Bengals,0,45,0,0,7,39,0,0,24,17,...,17,0,34,0,0,0,41,17,0,10
Bills,0,0,0,0,0,3,33,0,0,16,...,0,0,0,0,7,0,0,10,17,0
Broncos,0,0,12,0,0,27,0,0,37,68,...,42,0,7,17,0,0,10,0,0,0
Browns,0,6,27,6,6,0,0,0,23,41,...,23,0,3,0,0,0,27,0,0,20
Buccaneers,0,0,0,20,0,0,0,0,0,0,...,0,0,0,13,27,24,0,0,0,0
Cardinals,25,41,0,0,0,0,0,0,0,0,...,0,52,0,0,0,58,0,28,17,30
Chargers,0,0,27,0,55,30,0,0,0,80,...,48,0,26,23,0,0,28,0,42,0
Chiefs,0,0,10,10,57,34,0,0,21,0,...,26,0,24,14,0,0,27,0,0,0


We then apply `Laplace's rule of succession`.

In [3]:
def get_strength(df, i, j):
    s_ij = df.iloc[i].iloc[j]
    s_ji = df.iloc[j].iloc[i]
    s = (s_ij + 1) / (s_ij + s_ji + 2)
    return s
    
sscore_df = pd.DataFrame([[get_strength(rscore_df, i, j) for j in range(len(teams))] 
                          for i in range(len(teams))], index=teams, columns=teams)
sscore_df

,49ers,Bears,Bengals,Bills,Broncos,Browns,Buccaneers,Cardinals,Chargers,Chiefs,...,Raiders,Rams,Ravens,Redskins,Saints,Seahawks,Steelers,Texans,Titans,Vikings
49ers,0.500000,0.611111,0.500000,0.500000,0.500000,0.500000,0.500000,0.633803,0.500000,0.500000,...,0.500000,0.901408,0.500000,0.500000,0.500000,0.569444,0.500000,0.468085,0.444444,0.471698
Bears,0.388889,0.500000,0.192982,0.500000,0.500000,0.815789,0.500000,0.343750,0.500000,0.500000,...,0.500000,0.642857,0.200000,0.500000,0.500000,0.565217,0.545455,0.500000,0.500000,0.412281
Bengals,0.500000,0.807018,0.500000,0.500000,0.380952,0.588235,0.500000,0.500000,0.471698,0.620690,...,0.461538,0.500000,0.614035,0.500000,0.500000,0.500000,0.560000,0.382979,0.500000,0.261905
Bills,0.500000,0.500000,0.500000,0.500000,0.500000,0.363636,0.618182,0.500000,0.500000,0.607143,...,0.500000,0.500000,0.500000,0.500000,0.222222,0.500000,0.500000,0.255814,0.300000,0.500000
Broncos,0.500000,0.500000,0.619048,0.500000,0.500000,0.800000,0.500000,0.500000,0.404255,0.543307,...,0.641791,0.500000,0.205128,0.391304,0.500000,0.500000,0.275000,0.500000,0.500000,0.500000
Browns,0.500000,0.184211,0.411765,0.636364,0.200000,0.500000,0.500000,0.500000,0.436364,0.545455,...,0.705882,0.500000,0.072727,0.500000,0.500000,0.500000,0.451613,0.500000,0.500000,0.375000
Buccaneers,0.500000,0.500000,0.500000,0.381818,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.451613,0.333333,0.757576,0.500000,0.500000,0.500000,0.500000
Cardinals,0.366197,0.656250,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.688312,0.500000,0.500000,0.500000,0.710843,0.500000,0.568627,0.461538,0.632653
Chargers,0.500000,0.500000,0.528302,0.500000,0.595745,0.563636,0.500000,0.500000,0.500000,0.786408,...,0.569767,0.500000,0.457627,0.533333,0.500000,0.500000,0.426471,0.500000,0.704918,0.500000
Chiefs,0.500000,0.500000,0.379310,0.392857,0.456693,0.454545,0.500000,0.500000,0.213592,0.500000,...,0.529412,0.500000,0.390625,0.681818,0.500000,0.500000,0.528302,0.500000,0.500000,0.500000


Finally, we combat skew in the data.

In [4]:
import numpy as np

def h(x):
    h = 0.5 + ((np.sign(x - 0.5) * np.sqrt(np.abs(2 * x - 1))) / 2)
    return h

X = sscore_df.applymap(h)
X

,49ers,Bears,Bengals,Bills,Broncos,Browns,Buccaneers,Cardinals,Chargers,Chiefs,...,Raiders,Rams,Ravens,Redskins,Saints,Seahawks,Steelers,Texans,Titans,Vikings
49ers,0.500000,0.735702,0.500000,0.500000,0.500000,0.500000,0.500000,0.758653,0.500000,0.500000,...,0.500000,0.948000,0.500000,0.500000,0.500000,0.686339,0.500000,0.373677,0.333333,0.381042
Bears,0.264298,0.500000,0.108198,0.500000,0.500000,0.897360,0.500000,0.220492,0.500000,0.500000,...,0.500000,0.767261,0.112702,0.500000,0.500000,0.680579,0.650756,0.500000,0.500000,0.290573
Bengals,0.500000,0.891802,0.500000,0.500000,0.256025,0.710042,0.500000,0.500000,0.381042,0.745652,...,0.361325,0.500000,0.738783,0.500000,0.500000,0.500000,0.673205,0.258110,0.500000,0.154967
Bills,0.500000,0.500000,0.500000,0.500000,0.500000,0.238884,0.743086,0.500000,0.500000,0.731455,...,0.500000,0.500000,0.500000,0.500000,0.127322,0.500000,0.500000,0.150582,0.183772,0.500000
Broncos,0.500000,0.500000,0.743975,0.500000,0.500000,0.887298,0.500000,0.500000,0.281203,0.647151,...,0.766262,0.500000,0.116026,0.266874,0.500000,0.500000,0.164590,0.500000,0.500000,0.500000
Browns,0.500000,0.102640,0.289958,0.761116,0.112702,0.500000,0.500000,0.500000,0.321623,0.650756,...,0.820844,0.500000,0.037792,0.500000,0.500000,0.500000,0.344457,0.500000,0.500000,0.250000
Buccaneers,0.500000,0.500000,0.500000,0.256914,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.344457,0.211325,0.858870,0.500000,0.500000,0.500000,0.500000
Cardinals,0.241347,0.779508,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.806848,0.500000,0.500000,0.500000,0.824687,0.500000,0.685240,0.361325,0.757539
Chargers,0.500000,0.500000,0.618958,0.500000,0.718797,0.678377,0.500000,0.500000,0.500000,0.878423,...,0.686772,0.500000,0.354444,0.629099,0.500000,0.500000,0.308259,0.500000,0.820092,0.500000
Chiefs,0.500000,0.500000,0.254348,0.268545,0.352849,0.349244,0.500000,0.500000,0.121577,0.500000,...,0.621268,0.500000,0.266146,0.801511,0.500000,0.500000,0.618958,0.500000,0.500000,0.500000


## Numpy

We will use numpy to find the eigenvalues and eigenvectors.

In [5]:
e, E = np.linalg.eig(X)

In [6]:
e

array([1.58321633e+01+0.j        , 2.22768644e-03+1.85772142j,
       2.22768644e-03-1.85772142j, 3.39712442e-03+1.35674872j,
       3.39712442e-03-1.35674872j, 1.00041396e-03+1.26017375j,
       1.00041396e-03-1.26017375j, 1.04401936e-02+1.20532274j,
       1.04401936e-02-1.20532274j, 4.22054558e-03+1.07142414j,
       4.22054558e-03-1.07142414j, 7.62218544e-04+0.95584753j,
       7.62218544e-04-0.95584753j, 1.50585060e-02+0.7822698j ,
       1.50585060e-02-0.7822698j , 2.70582449e-03+0.72218668j,
       2.70582449e-03-0.72218668j, 2.01788728e-03+0.63803178j,
       2.01788728e-03-0.63803178j, 4.25439229e-03+0.57011899j,
       4.25439229e-03-0.57011899j, 1.11927138e-02+0.4517283j ,
       1.11927138e-02-0.4517283j , 7.65441825e-03+0.35245526j,
       7.65441825e-03-0.35245526j, 1.15967256e-02+0.24209874j,
       1.15967256e-02-0.24209874j, 3.16542655e-03+0.18988739j,
       3.16542655e-03-0.18988739j, 2.97621199e-03+0.07044387j,
       2.97621199e-03-0.07044387j, 2.49615168e-03+0.j  

The sorted eigenvector shows that the Saints are at the top with the Rams at the bottom. The Saints won the Super Bowl during the 2009-2010 season.

In [7]:
pd.Series(E[:,0], index=teams).sort_values(ascending=True)

Saints       -0.203355+0.000000j
Packers      -0.201014+0.000000j
Patriots     -0.197235+0.000000j
Chargers     -0.197097+0.000000j
Colts        -0.195922+0.000000j
Vikings      -0.195730+0.000000j
Cowboys      -0.195317+0.000000j
Jets         -0.195166+0.000000j
Eagles       -0.190664+0.000000j
Ravens       -0.190316+0.000000j
Steelers     -0.188673+0.000000j
Texans       -0.188029+0.000000j
Falcons      -0.183949+0.000000j
Cardinals    -0.182012+0.000000j
49ers        -0.179368+0.000000j
Broncos      -0.178877+0.000000j
Bengals      -0.177157+0.000000j
Panthers     -0.173233+0.000000j
Titans       -0.171839+0.000000j
Giants       -0.171517+0.000000j
Dolphins     -0.167714+0.000000j
Bears        -0.165493+0.000000j
Redskins     -0.163789+0.000000j
Bills        -0.163556+0.000000j
Jaguars      -0.162974+0.000000j
Chiefs       -0.157590+0.000000j
Browns       -0.157127+0.000000j
Seahawks     -0.153406+0.000000j
Raiders      -0.147552+0.000000j
Buccaneers   -0.147396+0.000000j
Lions     

## Scikit-Learn

Here, we use Scikit-Learn to retrieve the eigenvalues and eigenvectors.

In [8]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2, n_iter=100, random_state=37)
svd.fit(X)

TruncatedSVD(n_iter=100, random_state=37)

In [9]:
svd.singular_values_

array([16.16520004,  1.85991954])

Again, we see the Saints at the top.

In [10]:
pd.Series(svd.components_[0,:], index=teams).sort_values(ascending=True)

Saints        0.148945
Packers       0.151001
Patriots      0.154887
Chargers      0.155021
Colts         0.156045
Vikings       0.156115
Cowboys       0.156881
Jets          0.156972
Eagles        0.161209
Ravens        0.161649
Steelers      0.163317
Texans        0.163902
Falcons       0.167846
Cardinals     0.169756
49ers         0.172460
Broncos       0.172764
Bengals       0.174560
Panthers      0.178314
Titans        0.179604
Giants        0.180136
Dolphins      0.183847
Bears         0.186135
Redskins      0.187734
Bills         0.188043
Jaguars       0.188691
Chiefs        0.193845
Browns        0.194392
Seahawks      0.198091
Buccaneers    0.203681
Raiders       0.203764
Lions         0.207208
Rams          0.211215
dtype: float64

Here are the ratings side-by-side.

In [11]:
pd.DataFrame({
    'numpy': pd.Series(E[:,0], index=teams),
    'scikit': pd.Series(svd.components_[0,:], index=teams)
})

,numpy,scikit
49ers,-0.179368+0.000000j,0.172460
Bears,-0.165493+0.000000j,0.186135
Bengals,-0.177157+0.000000j,0.174560
Bills,-0.163556+0.000000j,0.188043
Broncos,-0.178877+0.000000j,0.172764
Browns,-0.157127+0.000000j,0.194392
Buccaneers,-0.147396+0.000000j,0.203681
Cardinals,-0.182012+0.000000j,0.169756
Chargers,-0.197097+0.000000j,0.155021
Chiefs,-0.157590+0.000000j,0.193845


Here's the ranking side-by-side. The only difference is with the Buccaneers and Raiders.

In [12]:
ranking_df = pd.DataFrame({
    'numpy': pd.Series(E[:,0], index=teams).sort_values().index,
    'scikit': pd.Series(svd.components_[0,:], index=teams).sort_values().index
})
ranking_df['differs?'] = ranking_df.numpy != ranking_df.scikit
ranking_df

,numpy,scikit,differs?
0,Saints,Saints,False
1,Packers,Packers,False
2,Patriots,Patriots,False
3,Chargers,Chargers,False
4,Colts,Colts,False
5,Vikings,Vikings,False
6,Cowboys,Cowboys,False
7,Jets,Jets,False
8,Eagles,Eagles,False
9,Ravens,Ravens,False


## Ranking by yards

Let's use yardage gains instead.

In [13]:
rscore_df = pd.DataFrame([[get_cummulative_score(df, t1, t2, 'yd1', 'yd2') if t1 != t2 else 0 for t2 in teams] 
              for t1 in teams], index=teams, columns=teams)
sscore_df = pd.DataFrame([[get_strength(rscore_df, i, j) for j in range(len(teams))] 
                          for i in range(len(teams))], index=teams, columns=teams)
X = sscore_df.applymap(h)

e, E = np.linalg.eig(X)

svd = TruncatedSVD(n_components=2, n_iter=100, random_state=37)
svd.fit(X)

ranking_df = pd.DataFrame({
    'numpy': pd.Series(E[:,0], index=teams).sort_values().index,
    'scikit': pd.Series(svd.components_[0,:], index=teams).sort_values().index
})
ranking_df['differs?'] = ranking_df.numpy != ranking_df.scikit
ranking_df

,numpy,scikit,differs?
0,Browns,Packers,True
1,Buccaneers,Cowboys,True
2,Raiders,Texans,True
3,Lions,Jets,True
4,Rams,Steelers,True
5,Chiefs,Eagles,True
6,Bills,Patriots,True
7,49ers,Vikings,True
8,Seahawks,Saints,True
9,Falcons,Ravens,True


## Turnovers

Let's rank the teams by turnovers. It's interesting that we can also rank teams to see who's the worst.

In [14]:
rscore_df = pd.DataFrame([[get_cummulative_score(df, t1, t2, 'to1', 'to2') if t1 != t2 else 0 for t2 in teams] 
              for t1 in teams], index=teams, columns=teams)
sscore_df = pd.DataFrame([[get_strength(rscore_df, i, j) for j in range(len(teams))] 
                          for i in range(len(teams))], index=teams, columns=teams)
X = sscore_df.applymap(h)

e, E = np.linalg.eig(X)

svd = TruncatedSVD(n_components=2, n_iter=100, random_state=37)
svd.fit(X)

ranking_df = pd.DataFrame({
    'numpy': pd.Series(E[:,0], index=teams).sort_values().index,
    'scikit': pd.Series(svd.components_[0,:], index=teams).sort_values().index
})
ranking_df['differs?'] = ranking_df.numpy != ranking_df.scikit
ranking_df

,numpy,scikit,differs?
0,Lions,Lions,False
1,Raiders,Raiders,False
2,Rams,Rams,False
3,Browns,Browns,False
4,Giants,Giants,False
5,Seahawks,Seahawks,False
6,Dolphins,Dolphins,False
7,Steelers,Steelers,False
8,Buccaneers,Buccaneers,False
9,Titans,Titans,False
